In [1]:
from os import listdir, chdir

In [2]:
import time

In [43]:
import pandas as pd
import numpy as np

import sqlite3

conn = sqlite3.connect('ETF.db')

c = conn.cursor()

In [4]:
# get into directory that has data in CSVs
chdir('./data')

In [5]:
def del_table(cursor, name):
    query = """DROP TABLE {}""".format(name)
    cursor.execute(query + ";")

In [6]:
def create_table(cursor, name):
    query = """CREATE TABLE {} (date text, open FLOAT, high FLOAT, low FLOAT, close FLOAT, adj_close FLOAT, volume INTEGER)""".format(name)
    cursor.execute(query + ";")

In [7]:
def insert(cursor, name, values):
    query = """INSERT INTO {} VALUES(?, ?, ?, ?, ?, ?, ?)""".format(name) + ";"
    
    cursor.execute(query, values)

In [33]:
def retrieve(c,name):
    query = """SELECT * FROM {}""".format(name) + ";"
    c.execute(query)
    
    rows = c.fetchall()
    
    for row in rows[:5]:
        print(row)

In [9]:
!pwd

/Users/Rich/Desktop/projects/stonk/ETF_scraper/data


In [ ]:
c.execute("""SELECT * FROM AAAU""")

In [ ]:
rows = c.fetchall()

In [ ]:
for row in rows[:5]:
    print(row)

# Script

In [ ]:
listdir()[0].split('.')[0]

In [19]:
pd.read_csv('AAAU.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-08-13,11.840000,11.840000,11.740000,11.820000,11.820000,508100
1,2018-08-20,11.880000,12.080000,11.850000,12.050000,12.050000,321800
2,2018-08-27,12.060000,12.130000,11.970000,12.000000,12.000000,1201300
3,2018-09-03,11.910000,12.070000,11.900000,11.960000,11.960000,1082200
4,2018-09-10,11.970000,12.120000,11.880000,11.930000,11.930000,92300
...,...,...,...,...,...,...,...
100,2020-07-13,18.070000,18.080000,17.889999,18.049999,18.049999,1492500
101,2020-07-20,18.120001,19.030001,18.059999,18.980000,18.980000,2855900
102,2020-07-27,19.320000,19.750000,19.260000,19.690001,19.690001,3417600
103,2020-08-03,19.639999,20.629999,19.540001,20.250000,20.250000,3365600


In [44]:
#alphabetically sort directory files and skip the first element which would be a .DS_STORE file
files = sorted(listdir())

## Script for parsing thru CSVs and insert into Database
for file in files[1:]:
    
    #read in data
    if file.split('.')[1] == 'csv':
        df = pd.read_csv(file)
    else:
        continue
    #get the name of the ETF
    name = file.split('.')[0]

###
#Completely clear the DB of all ETFs
#     try:
#         del_table(c,name)
#     except:
#         continue

    create_table(c,name)
#     conn.commit
###

#Go through each of the dataframes and insert each line into the db
    for i in range(len(df)):
        data = df.iloc[i]
        
        insert(c,name,tuple(data))
    
#after every table, commit and wait 1 second
    conn.commit
    time.sleep(1)

# #redefine name to None to make it not be confused in the next iteration
#     name = None

In [ ]:
#Check to see if we were able to store the data
for file in files[1:6]:
    print(retrieve(c,'AAAU'))

In [ ]:
retrieve(c,'AAAU.csv'.split('.')[0])

In [ ]:
#data is inserting the same data over and over. Need to fix script

The last volume column returns as bytes. To convert need to use from_bytes method when parsing data

In [34]:
#Check to see if we were able to store the data
for file in files[1:6]:
    data = retrieve(c,file.split('.')[0])

('2018-08-13', 11.84, 11.84, 11.74, 11.82, 11.82, b'\xc4\xc0\x07\x00\x00\x00\x00\x00')
('2018-08-20', 11.88, 12.08, 11.85, 12.05, 12.05, b'\x08\xe9\x04\x00\x00\x00\x00\x00')
('2018-08-27', 12.06, 12.13, 11.97, 12.0, 12.0, b'\x94T\x12\x00\x00\x00\x00\x00')
('2018-09-03', 11.91, 12.07, 11.9, 11.96, 11.96, b'X\x83\x10\x00\x00\x00\x00\x00')
('2018-09-10', 11.97, 12.12, 11.88, 11.93, 11.93, b'\x8ch\x01\x00\x00\x00\x00\x00')
('2010-07-19', 25.1, 25.540001, 24.700001, 25.280001000000002, 23.891865, b'\x04\n\x01\x00\x00\x00\x00\x00')
('2010-07-26', 25.4, 25.4, 24.99, 25.1, 23.72175, b'\x08\x84\x00\x00\x00\x00\x00\x00')
('2010-08-02', 25.700001, 25.91, 25.309998999999998, 25.709999, 24.298254, b'\xbcM\x00\x00\x00\x00\x00\x00')
('2010-08-09', 26.0, 26.23, 25.040001, 25.17, 23.787901, b' \x1c\x00\x00\x00\x00\x00\x00')
('2010-08-16', 25.280001000000002, 25.9, 25.24, 25.299999, 23.910763, b'\x18.\x00\x00\x00\x00\x00\x00')
('2010-07-19', 53.860001000000004, 56.080002, 53.669998, 55.939999, 46.317833

In [39]:
# type(int.from_bytes(rows[0][-1],byteorder = 'little'))

In [40]:
# for row in rows[:5]:
#     print(int.from_bytes(row[-1], byteorder = 'little'))

In [45]:
conn.commit()

In [46]:
conn.close()